In [26]:
import array
import random
import json

import numpy

from math import sqrt

from deap import algorithms
from deap import base
from deap import benchmarks
from deap.benchmarks.tools import diversity, convergence, hypervolume
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(1.0,-1.0,1.0))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox  = base.Toolbox()

toolbox.register("evaluate",evaluate)
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, 100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate(individual):
    f1 = sum(individual[1::2])
    f2 = sum(individual[::2])
    f3 = individual[0]*10
    return f1,f2,f3
toolbox.register("evaluate", evaluate)
toolbox.register("select", tools.selNSGA2)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)



def main(seed=None):
    random.seed(seed)

    NGEN = 250
    MU = 100
    CXPB = 0.9

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    # stats.register("avg", numpy.mean, axis=0)
    # stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)
    
    logbook = tools.Logbook()
    logbook.header = "gen", "evals", "std", "min", "avg", "max"
    
    pop = toolbox.population(n=MU)

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # This is just to assign the crowding distance to the individuals
    # no actual selection is done
    pop = toolbox.select(pop, len(pop))
    
    record = stats.compile(pop)
    logbook.record(gen=0, evals=len(invalid_ind), **record)
    print(logbook.stream)

    # Begin the generational process
    for gen in range(1, NGEN):
        # Vary the population
        offspring = tools.selTournamentDCD(pop, len(pop))
        offspring = [toolbox.clone(ind) for ind in offspring]
        
        for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
            if random.random() <= CXPB:
                toolbox.mate(ind1, ind2)
            
            toolbox.mutate(ind1)
            toolbox.mutate(ind2)
            del ind1.fitness.values, ind2.fitness.values
        
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Select the next generation population
        pop = toolbox.select(pop + offspring, MU)
        record = stats.compile(pop)
        logbook.record(gen=gen, evals=len(invalid_ind), **record)
        print(logbook.stream)

    print("Final population hypervolume is %f" % hypervolume(pop, [11.0, 11.0]))

    return pop, logbook
        
if __name__ == "__main__":
    # with open("pareto_front/zdt1_front.json") as optimal_front_data:
    #     optimal_front = json.load(optimal_front_data)
    # Use 500 of the 1000 points in the json file
    # optimal_front = sorted(optimal_front[i] for i in range(0, len(optimal_front), 2))
    
    pop, stats = main()
    # pop.sort(key=lambda x: x.fitness.values)
    
    # print(stats)
    # print("Convergence: ", convergence(pop, optimal_front))
    # print("Diversity: ", diversity(pop, optimal_front[0], optimal_front[-1]))
    
    # import matplotlib.pyplot as plt
    # import numpy
    
    # front = numpy.array([ind.fitness.values for ind in pop])
    # optimal_front = numpy.array(optimal_front)
    # plt.scatter(optimal_front[:,0], optimal_front[:,1], c="r")
    # plt.scatter(front[:,0], front[:,1], c="b")
    # plt.axis("tight")
    # plt.show()

/Users/puca/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/puca/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	evals	std	min             	avg	max             
0  	100  	   	[ 15.  15.   0.]	   	[ 33.  34.  10.]
1  	100  	   	[ 17.  15.   0.]	   	[ 35.  32.  10.]
2  	100  	   	[ 20.  15.   0.]	   	[ 37.  29.  10.]
3  	100  	   	[ 21.  14.   0.]	   	[ 37.  28.  10.]
4  	100  	   	[ 21.  11.   0.]	   	[ 40.  28.  10.]
5  	100  	   	[ 21.  10.   0.]	   	[ 40.  26.  10.]
6  	100  	   	[ 18.   9.   0.]	   	[ 40.  25.  10.]
7  	100  	   	[ 23.   9.   0.]	   	[ 41.  23.  10.]
8  	100  	   	[ 21.   9.   0.]	   	[ 41.  22.  10.]
9  	100  	   	[ 20.   7.   0.]	   	[ 45.  22.  10.]
10 	100  	   	[ 20.   6.   0.]	   	[ 45.  22.  10.]
11 	100  	   	[ 20.   6.   0.]	   	[ 45.  21.  10.]
12 	100  	   	[ 19.   6.   0.]	   	[ 45.  21.  10.]
13 	100  	   	[ 19.   6.   0.]	   	[ 45.  22.  10.]
14 	100  	   	[ 24.   6.   0.]	   	[ 45.  21.  10.]
15 	100  	   	[ 24.   5.   0.]	   	[ 45.  21.  10.]
16 	100  	   	[ 24.   5.   0.]	   	[ 45.  21.  10.]
17 	100  	   	[ 28.   5.   0.]	   	[ 45.  20.  10.]
18 	100  	  

TypeError: Reference point is not of same dimensionality as point set

In [7]:
def evaluate(individual):
    f1 = sum(individual[1::2])
    f2 = sum(individual[::2])
    f3 = individual[0]*100
    return f1,f2,f3
toolbox.register("evaluate",evaluate)
toolbox.eval([1,2,3,4,5])


(6, 9, 100)

In [23]:
class Agent:
    def __init__(self):
        self.sex = "male"
    def get(self):
        print("a")
a = Agent()
a.get()

a


In [13]:
class a:
    def p(self):
        pass
i  =  a()

In [14]:
i.aaa

AttributeError: 'a' object has no attribute 'aaa'